In [12]:
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains.summarize import load_summarize_chain
import streamlit as st
from langchain.docstore.document import Document
import os
from dotenv import load_dotenv
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

In [ ]:
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [14]:
model=ChatGroq(model="gemma2-9b-it")

In [15]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\generativeai\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import assemblyai as aai

# Set the API key
aai.settings.api_key = "dadbbd65a132428bb4cbde4a36663f9f"  # Replace with your AssemblyAI API key

# Initialize the transcriber
transcriber = aai.Transcriber()

# Path to the audio file
audio_file = "./Audio.mp3"  # Replace with your actual file path

# Configure transcription settings
config = aai.TranscriptionConfig(speaker_labels=True)

transcript = transcriber.transcribe(audio_file, config)

if transcript.status == aai.TranscriptStatus.error:
    print(f"Transcription failed: {transcript.error}")
    exit(1)



In [18]:
transcript_text=transcript.text
docs = [Document(page_content=transcript_text)]
docs

[Document(metadata={}, page_content="Then I can do it. Otherwise you have. I will not continue for that. They have some training before they have. I have. I got certificate for that. It is a project genie AI. Okay. So can you just confirm once like here your enrollment is like a full stack development. Is that right? Yeah, yeah. Full stack. Yes, full stack. Actually I want I requested for these courses. But before that I have to register myself and I will go all towards instructions how to after I will take certificate for that or not. Okay. What team they decided for that? Please. But I have received everything in my email. From actually sir. From actually sir. Okay. So currently where are you living in now? I am here in India. In Kashmir. In Kashmir. You are. Okay. So is that like. Are you comfortable with the Indian timings? Everything right? Fine. Right. So are you like available or in any kind of timing which we see, we even. We are trying to connect you. But we couldn't be able t

In [19]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000019EE1588CE0>, search_kwargs={})

In [ ]:
# chunks_prompt="""
# Please summarize the below text:
# Speech:`{text}'
# Summary:
# """
# map_prompt_template=PromptTemplate(input_variables=['text'],
#                                     template=chunks_prompt)

In [ ]:
system_prompt=(
    "You are an AI trained assistant to do a call audit and analyze conversations between a salesperson and a customer."
    "Your task is to focus specifically on the points made by the salesperson to persuade the customer."
    "After reading the conversation, summarize the persuasive points from the salesperson in bullet points," 
    "highlighting the arguments, benefits, or strategies used by the salesperson to convince the customer and also highlight where salesperson was"
    "not able to convince the customer"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [21]:
question_answer_chain=create_stuff_documents_chain(model,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [22]:
response=rag_chain.invoke({"input":"can u give me the conversations where sales person try to convince with customer"})
response['answer']

"Here are the persuasive points made by the salesperson, focusing on arguments, benefits, and strategies:\n\n* **Offers flexibility:**  The salesperson tries to find a convenient time for the customer by asking about their availability and suggesting a specific window.\n* **Provides multiple contact methods:** The salesperson offers both phone and email contact, emphasizing their commitment to connecting.\n* **Highlights the value of the course:** The salesperson mentions the course's comprehensive nature and the high demand for it, implying its prestige and effectiveness.\n* **Addresses concerns and offers solutions:**\n    * The salesperson acknowledges the customer's hesitation about the main class and suggests attending a pre-boot class to experience it firsthand.\n    * They also address the customer's potential EMI issues by stating they have various options and are willing to work with them.\n* **Creates a sense of urgency:** The salesperson encourages the customer to attend the